In [3]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import numpy as np
import os, glob, cv2, torch
from sklearn.metrics.pairwise import cosine_similarity

# Setup environment and device configuration
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")


# data_dir = './videos/MRI515_T2'
# video_dir = os.path.join(data_dir, 'img_in_png_SAM_L4-L5_4class_T2')
# target_dir = os.path.join(data_dir, 'img_in_png_SAM_L4-L5_4class_T2_DINO_order')

data_dir = './videos/CT_tt'
video_dir = os.path.join(data_dir, 'img_in_jpg')
target_dir = os.path.join(data_dir, 'img_in_jpg_2class_DINO_order')


# # data_dir = './videos/MRI515_T1/data_in_jpg_raw_name'
# # video_dir = os.path.join(data_dir, 'img_in_png_SAM_L4-L5_4class_T1')
# # target_dir = os.path.join(data_dir, 'img_in_png_SAM_L4-L5_4class_T1_DINO_order')


# # data_dir = 'videos/AFL_MRI/data_in_nii'
# data_dir = 'videos/bed_rest_MRI/data_in_nii'
# level = 'L5low'

# video_dir = os.path.join(data_dir, 'img_in_jpg_' + level  + '_4class')
# target_dir = os.path.join(data_dir, 'img_in_jpg_' + level  + '_4class_DINO_order')

os.makedirs(target_dir, exist_ok=True)

# Load pre-trained DINO model and processor
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
model = AutoModel.from_pretrained('facebook/dinov2-large')
model.to(device)
model.eval()

# Function to load image and prepare it for the model
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    return inputs

# Function to compute image embeddings
def get_image_embedding(image_path):
    inputs = process_image(image_path)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()



In [4]:
# 收集所有图片文件
all_image_files = sorted(glob.glob(os.path.join(video_dir, "**/*.png"), recursive=True) +
                         glob.glob(os.path.join(video_dir, "**/*.jpg"), recursive=True))

# 计算所有图片的特征向量
image_embeddings = []
for image_path in all_image_files:
    embedding = get_image_embedding(image_path)
    image_embeddings.append(embedding)

# 计算相似度矩阵
image_embeddings = np.vstack(image_embeddings)
similarity_matrix = cosine_similarity(image_embeddings)

# 找到总相似度最高的图片作为第一个
similarity_sums = similarity_matrix.sum(axis=1)
first_image_index = np.argmax(similarity_sums)  # 总相似度最高的图片索引

# 按相似度从第一个图片开始重新排序
sorted_indices = [first_image_index]
remaining_indices = set(range(len(all_image_files))) - {first_image_index}

while remaining_indices:
    last_index = sorted_indices[-1]
    next_index = max(remaining_indices, key=lambda x: similarity_matrix[last_index, x])
    sorted_indices.append(next_index)
    remaining_indices.remove(next_index)

# 保存重新排序的图片
for i, idx in enumerate(sorted_indices):
    image_path = all_image_files[idx]
    image = cv2.imread(image_path)
    output_path = os.path.join(target_dir, f"{i:05d}_{os.path.basename(image_path)}")
    print(i, idx, output_path, '\n\n')
    
    cv2.imwrite(output_path, image)

print(f"重新排序的图片已保存到 {target_dir}")

0 261 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00000_s1377_img_L4L5_slice148.jpg 


1 135 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00001_s0703_img_L4L5_slice147.jpg 


2 143 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00002_s0733_img_L4L5_slice140.jpg 


3 254 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00003_s1363_img_L4L5_slice104.jpg 


4 86 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00004_s0472_img_L4L5_slice139.jpg 


5 7 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00005_s0028_img_L4L5_slice121.jpg 


6 1 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00006_s0011_img_L4L5_slice140.jpg 


7 12 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00007_s0040_img_L4L5_slice137.jpg 


8 127 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00008_s0667_img_L4L5_slice156.jpg 


9 273 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00009_s1397_img_L4L5_slice120.jpg 


10 20 ./videos/CT_tt/img_in_jpg_2class_T2_DINO_order/00010_s0072_img_L4L5_slice122.jpg 


11 79 ./videos/C

In [13]:
import os
import shutil
import glob

# 定义文件夹路径
import re

def modify_filename(base_name):
    # 替换 img 为 label
    new_name = re.sub(r'img', 'label', base_name)
    # 替换 .jpg 为 .png（只处理结尾）
    new_name = re.sub(r'\.jpg$', '.png', new_name)
    return new_name


# video_dir = os.path.join(data_dir, 'img_in_jpg_' + level  + '_4class_DINO_order')
# label_dir  =  os.path.join(data_dir, 'label_in_png_' + level  + '_4class')
# target_label_dir = os.path.join(data_dir, 'label_in_png_' + level  + '_4class_DINO_order')

label_dir  =  os.path.join(data_dir, 'label_in_png')
target_label_dir = os.path.join(data_dir, 'label_in_png_2class_DINO_order')

os.makedirs(target_label_dir, exist_ok=True)

# 获取视频文件列表
video_files = sorted(glob.glob(os.path.join(video_dir, "*.jpg")))

# 遍历文件名提取ID，并处理对应的标签文件
for video_file in video_files:
    # 获取文件名（如 "00001_GL_106_slice_016_img_.jpg"）
    base_name = os.path.basename(video_file)
    
    # 提取ID部分（如 "GL_106"）
    parts = base_name.split('_')
    id_part = "_".join(parts[1:3])  # 提取 "GL_106"
    if 'CT_tt' in label_dir:
        parts = base_name.split('_')
        id_part = "_".join(parts[0:1])  # 提取 "GL_106"
    print('pid',id_part)
    # 查找标签文件
    label_file_pattern = os.path.join(label_dir, f"*{id_part}*.png")
    label_files = glob.glob(label_file_pattern)
    
    if len(label_files) == 0:
        print(f"未找到匹配的标签文件: {id_part}")
        continue
    
    # 如果找到多个匹配，提示警告（根据需要可以调整逻辑）
    if len(label_files) > 1:
        print(f"找到多个匹配的标签文件: {label_files}")
    
    # 取第一个匹配的文件
    label_file = label_files[0]
    
    # 生成目标文件名，保持与视频文件一致的格式
    target_label_file = os.path.join(target_label_dir, base_name.replace('img.jpg', 'seg_.png'))
    if 'CT_tt' in label_dir:
        target_label_file = os.path.join(target_label_dir, modify_filename(base_name))
    # 复制文件到目标文件夹
    shutil.copy(label_file, target_label_file)

    print(f"处理完成: {label_file} -> {target_label_file}")
    print('\n\n\n')

print(f"所有标签文件已保存到 {target_label_dir}")


pid s0010
处理完成: ./videos/CT_tt/label_in_png/s0010_png_L4L5_slice130.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/s0010_label_L4L5_slice130.png




pid s0011
处理完成: ./videos/CT_tt/label_in_png/s0011_png_L4L5_slice140.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/s0011_label_L4L5_slice140.png




pid s0014
处理完成: ./videos/CT_tt/label_in_png/s0014_png_L4L5_slice135.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/s0014_label_L4L5_slice135.png




pid s0015
处理完成: ./videos/CT_tt/label_in_png/s0015_png_L4L5_slice147.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/s0015_label_L4L5_slice147.png




pid s0016
处理完成: ./videos/CT_tt/label_in_png/s0016_png_L4L5_slice142.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/s0016_label_L4L5_slice142.png




pid s0022
处理完成: ./videos/CT_tt/label_in_png/s0022_png_L4L5_slice130.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/s0022_label_L4L5_slice130.png




pid s0024
处理完成: ./videos/CT_tt/label_in_png/s0024_png_L4L5_slice185.pn

In [14]:
import os
import glob

# 定义文件夹路径
# data_dir = './videos/AFL_MRI/AFL_L4L5_in_JPG_2class_DINO'

label_dir = target_label_dir 

# 获取所有视频文件和标签文件
video_files = sorted(glob.glob(os.path.join(video_dir, "*.jpg")))
label_files = sorted(glob.glob(os.path.join(label_dir, "*.png")))

# 检查数量是否一致
if len(video_files) != len(label_files):
    print(f"文件数量不匹配: 视频文件({len(video_files)}) 与 标签文件({len(label_files)})")
else:
    print(f"文件数量匹配: {len(video_files)} 个文件")

# 提取文件名中的 ID
def extract_id(file_name):
    base_name = os.path.basename(file_name)
    parts = base_name.split('_')
    return "_".join(parts[1:3])  # 提取 ID，如 "GL_106"

# 对比两个文件夹的 ID
mismatch_count = 0
for video_file, label_file in zip(video_files, label_files):
    video_id = extract_id(video_file)
    label_id = extract_id(label_file)
    print(video_id, label_id)
    
    if video_id != label_id:
        print(f"不匹配: 视频文件 {video_file} 的 ID ({video_id}) 与 标签文件 {label_file} 的 ID ({label_id}) 不一致")
        mismatch_count += 1

# 总结结果
if mismatch_count == 0:
    print("所有文件的 ID 均匹配！")
else:
    print(f"总共有 {mismatch_count} 个文件的 ID 不匹配。")


文件数量匹配: 288 个文件
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0010_img_L4L5_slice130.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0010_label_L4L5_slice130.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0011_img_L4L5_slice140.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0011_label_L4L5_slice140.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0014_img_L4L5_slice135.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0014_label_L4L5_slice135.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0015_img_L4L5_slice147.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0015_label_L4L5_slice147.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0016_img_L4L5_slice142.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0016_label